In [1]:
# Imports
from pprint import pprint
import json
import pandas as pd

In [2]:
gps_data = []
json_data = {}
with open('gps_dump.json') as file:
    for line in file:
        parts = line.split(' ',1) 
        if '/gps/rmc' in parts[0]:
            gps_data.append(parts)
#             print(parts)
#             break
#         else:
#             json_data[parts[0].replace('owntracks/','')] = json.loads(parts[1])
# pprint(gps_data[:5])
# pprint(json_data['sparvagn/417'])

In [104]:
# process json location
# data = []
# for i, key in enumerate(json_data.keys()):
#     data.append((key,json_data[key]['_type'],json_data[key]['lat'],
#                json_data[key]['lon'],
#                json_data[key]['tst']
#                ))
# df_json = pd.DataFrame(data=data, columns=['vehicle','type','lat','lon','timestamp'])

In [105]:
# df_json['location (latitude, longitude)'] = df_json[['lat','lon']].apply(lambda x: tuple(x),axis=1)
# df_json['timestamp'] = df_json['timestamp'].apply(lambda x: datetime.fromtimestamp(x))

# # df_json = df_json.drop(columns=['type','timestamp','location (latitude, longitude)'])
# display(df_json[df_json['vehicle']=='sparvagn/303'])
# df_json.to_csv(path_or_buf='locations.csv',index=False)

,vehicle,type,lat,lon,timestamp,"location (latitude, longitude)"
236,sparvagn/303,location,57.751147,12.071321,2019-05-10 20:48:07,"(57.75114666666666, 12.0713215)"


In [60]:
# vehicles = df_json.groupby('vehicle')
# 57.751147
# 12.071321
# # for name,group in vehicles:
#     print(name)
#     display(group)

In [63]:
# pprint(gps_data[:1])
# pprint(json_data['sparvagn/417'])

[['owntracks/sparvagn/417/gps/rmc',
  '$GPRMC,184431.00,A,5742.47961,N,01159.73621,E,0.002,,100519,,,A*71\n']]
{'_type': 'location',
 'lat': 57.707956,
 'lon': 11.994742833333333,
 'tst': 1557514087}


In [4]:
data_gps = []
import pynmea2
for item in gps_data:
    data_list = item[1].split(',')
#     print(len(data_list))
#     if len(data_list) < 10:
#         print(data_list)
    msg = pynmea2.parse(item[1])
#     print(msg.longitude)
#     breakabs
    try:
        data_gps.append((item[0].replace('owntracks/','').replace('/gps/rmc',''),
                        data_list[9],
                        msg.timestamp,
                        data_list[2],
                        msg.latitude,
                        msg.longitude))
    except:
        pass
df_gps = pd.DataFrame(data=data_gps, columns=['vehicle','date','time','active or void','lat','lon'])
# display(df_gps[df_gps['vehicle']=='sparvagn/303'])

In [28]:
# sparvagn = df_gps[df_gps['vehicle']=='sparvagn/415'].iloc[::3] #[df_gps['vehicle']=='sparvagn/303']
df_gps.to_csv(path_or_buf='gps_data.csv',index=False)

In [40]:
trams_with_connection_loss = df_gps[df_gps['active or void'] == 'V']
# display(trams_with_connection_loss)
trams = df_gps.groupby('vehicle')
dict_trams = {}

for name,group in trams:
    tram_locations = []
    
    conn = None
    pre_loc = None
    for idx, row in group.iterrows():
#         print(row)
        if conn is None:
            conn = row['active or void']
            pre_loc = (row['lat'], row['lon'])
        elif conn == 'A' and row['active or void'] == 'V':
            tram_locations.append(pre_loc)
            conn = row['active or void']
        elif conn == row['active or void'] and conn == 'A':
            conn = row['active or void']
            pre_loc = (row['lat'], row['lon'])
        elif conn == 'V' and row['active or void'] == 'A':
            conn = row['active or void']
            pre_loc = (row['lat'], row['lon'])
            tram_locations.append(pre_loc)
        else:
            pass
    if  tram_locations:
        dict_trams[name] = tram_locations
        
pprint(dict_trams)
# with open('tram_lost_locations.json', 'w') as json_file:  
#     json.dump(dict_trams, json_file)


{'sparvagn/301': [(57.692002333333335, 11.990103666666666),
                  (57.688943333333334, 11.969925666666667)],
 'sparvagn/302': [(57.6923, 11.9899745), (57.6904425, 11.9747785)],
 'sparvagn/305': [(57.6512395, 11.907624666666667),
                  (57.65341183333334, 11.911094666666667)],
 'sparvagn/310': [(57.744601833333334, 12.029503833333333),
                  (57.7462795, 12.0291835),
                  (57.7490365, 12.030772166666667),
                  (57.749513, 12.0305185),
                  (57.749654, 12.030584),
                  (57.752263, 12.032768666666666),
                  (57.761136, 12.042635833333334),
                  (57.76131433333333, 12.04825),
                  (57.75978683333334, 12.053847166666667),
                  (57.759640833333336, 12.057193),
                  (57.759640833333336, 12.057193),
                  (57.75956816666667, 12.060849333333334),
                  (57.759147166666665, 12.0625645),
                  (57.7563588333333

In [42]:
t = (1,2,3)
t = (1,) + t
print(t)

(1, 1, 2, 3)


In [43]:
tram_locations = []
for name,group in trams:
    conn = None
    pre_loc = None
    for idx, row in group.iterrows():
#         print(row)
        if conn is None:
            conn = row['active or void']
            pre_loc = (row['lat'], row['lon'])
        elif conn == 'A' and row['active or void'] == 'V':
            pre_loc = (0,) + pre_loc
            tram_locations.append(pre_loc)
            conn = row['active or void']
        elif conn == row['active or void'] and conn == 'A':
            conn = row['active or void']
            pre_loc = (row['lat'], row['lon'])
        elif conn == 'V' and row['active or void'] == 'A':
            conn = row['active or void']
            pre_loc = (1, row['lat'], row['lon'])
            tram_locations.append(pre_loc)
        else:
            pass
pprint(tram_locations)

[(0, 57.692002333333335, 11.990103666666666),
 (1, 57.688943333333334, 11.969925666666667),
 (0, 57.6923, 11.9899745),
 (1, 57.6904425, 11.9747785),
 (0, 57.6512395, 11.907624666666667),
 (1, 57.65341183333334, 11.911094666666667),
 (0, 57.744601833333334, 12.029503833333333),
 (1, 57.7462795, 12.0291835),
 (0, 57.7490365, 12.030772166666667),
 (1, 57.749513, 12.0305185),
 (0, 57.749654, 12.030584),
 (1, 57.752263, 12.032768666666666),
 (0, 57.761136, 12.042635833333334),
 (1, 57.76131433333333, 12.04825),
 (0, 57.75978683333334, 12.053847166666667),
 (1, 57.759640833333336, 12.057193),
 (0, 1, 57.759640833333336, 12.057193),
 (1, 57.75956816666667, 12.060849333333334),
 (0, 57.759147166666665, 12.0625645),
 (1, 57.75635883333333, 12.0670045),
 (0, 57.74437266666666, 12.029786666666666),
 (1, 57.74603183333333, 12.0297505),
 (0, 57.74894183333333, 12.0307455),
 (1, 57.752321333333335, 12.032958333333333),
 (0, 57.752375666666666, 12.032961),
 (1, 57.752574, 12.032995833333333),
 (0, 57

In [45]:
with open('tram_lost_locations.csv', 'w') as f:  
    for item in tram_locations:
        f.write(str(item[0])+','+str(item[1])+','+str(item[2])+'\n')

In [22]:
display(df_gps[(df_gps['vehicle'] == 'sparvagn/329') & (df_gps['active or void'] == 'V')])


,vehicle,date,time,active or void,lat,lon
132389,sparvagn/329,110519,10:11:10,V,0.0,0.0
132667,sparvagn/329,110519,10:11:11,V,0.0,0.0
132955,sparvagn/329,110519,10:11:12,V,0.0,0.0
133204,sparvagn/329,110519,10:11:13,V,0.0,0.0
133522,sparvagn/329,110519,10:11:14,V,0.0,0.0
134369,sparvagn/329,110519,10:11:15,V,0.0,0.0
134370,sparvagn/329,110519,10:11:16,V,0.0,0.0
134419,sparvagn/329,110519,10:11:17,V,0.0,0.0
134542,sparvagn/329,110519,10:11:18,V,0.0,0.0
134806,sparvagn/329,110519,10:11:19,V,0.0,0.0


In [24]:
# sparvagn = df_gps[df_gps['vehicle']=='sparvagn/329'].iloc[::3] #[df_gps['vehicle']=='sparvagn/303']
trams_with_connection_loss.to_csv(path_or_buf='trams_with_connection_loss.csv',index=False)

In [25]:
conn = None


vehicle           sparvagn/421
date                    110519
time                  10:03:13
active or void               V
lat                          0
lon                          0
Name: 19, dtype: object
